In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch
import torchmetrics
import torch.nn as nn
import torch.nn.functional as F
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.model_selection import train_test_split
from torchmetrics import MeanAbsoluteError
from datetime import datetime
import gc

#### 1. Baseline LSTM Model without tuning, 
####    Because i misread the evaluation section i have modeled expiratory phase too, this version is modified for inspiratory phase only - modified custom loss function

In [ ]:
X = pd.read_csv('/kaggle/input/ventilator-pressure-prediction/train.csv')
X_test = pd.read_csv('/kaggle/input/ventilator-pressure-prediction/test.csv')
X.pop('id')
X_test.pop('id')
y = X['pressure']
X.pop('pressure')


In [ ]:
X['u_in_cumsum'] = (X['u_in']).groupby(X['breath_id']).cumsum()
X_test['u_in_cumsum']  = (X_test['u_in']).groupby(X_test['breath_id']).cumsum()
# X['u_out_cumsum'] = (X['u_out']).groupby(X['breath_id']).cumsum()
# X_test['u_out_cumsum']  = (X_test['u_out']).groupby(X_test['breath_id']).cumsum()
# X['RC']= X['u_in']*(1-X['u_out'])*(1-np.exp(-X['time_step']/(1e-3*X['R']*X['C'])))
# X_test['RC']= X_test['u_in']*(1-X_test['u_out'])*(1-np.exp(-X_test['time_step']/(X_test['R']*X_test['C'])))

X.pop('breath_id')
X_test.pop('breath_id')

In [ ]:
# pd.set_option('display.max_rows',100)
# X.iloc[:100]

In [ ]:
# X_off=X.iloc[:400,:].copy()
# y_off=y.iloc[:400].copy()
# X=X.iloc[400:,:]
# y=y.iloc[400:]

In [ ]:
X_tmp = X.iloc[:,4].copy()
X_tmp_test = X_test.iloc[:,4].copy()
# X_tmp_off = X_off.iloc[:,4].copy()
st = StandardScaler()
X = st.fit_transform(X)
X_test = st.transform(X_test)
# X_off = st.transform(X_off)
X[:,4]=np.array(X_tmp)
X_test[:,4]=np.array(X_tmp_test)
# X_off[:,4]=np.array(X_tmp_off)

In [ ]:
X=np.float32(X).reshape(-1,80,6)
X_test = np.float32(X_test).reshape(-1,80,6)
y=np.float32(y).reshape(-1,80,1)

# X_off=np.float32(X_off).reshape(-1,80,6)
# y_off=np.float32(y_off).reshape(-1,80,1)

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, n_dim, n_hdim=30, n_layers=1, v_dropout=0):
        super().__init__()
        self.n_dim = n_dim
        self.n_hdim = n_hdim
        self.n_layers = n_layers
        self.v_dropout = v_dropout
        self.lstm = nn.LSTM(
        input_size=128,
        hidden_size=self.n_hdim,
        num_layers=self.n_layers,
        dropout = self.v_dropout,
        bidirectional=True,
        batch_first=True)
        self.inp = nn.Linear(self.n_dim,64)
        self.inp2 = nn.Linear(64,128)
        self.fc = nn.Linear(self.n_hdim*2,64)#2380 for 20
        self.out = nn.Linear(64,1)


        torch.nn.init.xavier_normal_(self.out.weight)
        torch.nn.init.xavier_normal_(self.fc.weight)
        torch.nn.init.xavier_normal_(self.inp.weight)
        torch.nn.init.xavier_normal_(self.inp2.weight)


    def forward(self, x):
        x = F.relu(self.inp(x))
        x = F.relu(self.inp2(x))
        x, _ = self.lstm(x)

        x = F.relu(self.fc(x))
        x = F.relu(self.out(x))

        return x

In [ ]:
class L1Loss_masked(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, preds, y, u_out):


        mask = 1 - u_out
        mae = torch.abs(mask * (y - preds))
        mae = torch.sum(mae) / torch.sum(mask)

        return mae

In [ ]:
def batch_gd(X, X_test, y, epochs, n_folds):


    gc.collect()
    kfold = KFold(n_splits = n_folds, random_state=42, shuffle=True)
    y_pred = torch.zeros(4024000,1)

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
       # device = 'cpu'
    print('Using {} device'.format(device))   
    y_pred = y_pred.to(device)
    xtest = torch.from_numpy(X_test)
    test_loader = torch.utils.data.DataLoader(xtest, batch_size=32, shuffle=False)


    for idx in kfold.split(X=X, y=y):
        best_loss = 1e4
        train_idx, val_idx = idx[0], idx[1]
        xtrain = X[train_idx]
        ytrain = y[train_idx]
        xval = X[val_idx]
        yval = y[val_idx]
        
        ytrain = np.array(ytrain)
        yval = np.array(yval)
        xtrain = torch.from_numpy(xtrain)
        xval = torch.from_numpy(xval)
        ytrain = torch.from_numpy(ytrain)
        yval = torch.from_numpy(yval)

        
        train_dataset = torch.utils.data.TensorDataset(xtrain,ytrain)
        train_loader=torch.utils.data.DataLoader(train_dataset, batch_size=256, shuffle=True, num_workers=4 )
 
        val_dataset = torch.utils.data.TensorDataset(xval,yval)
        val_loader=torch.utils.data.DataLoader(val_dataset, batch_size=256, shuffle=False , num_workers=4)
        
        model = LSTMModel(6,128,3)

        criterion = L1Loss_masked()
        optimizer = torch.optim.Adam(model.parameters(), lr=0.0008, eps=1e-08)
        device = 'cuda' if torch.cuda.is_available() else 'cpu'

        model.to(device)
        gc.collect()
        for epoch in range(epochs):
            model.train()
            t0 = datetime.now()
            train_loss = []
            for inputs, targets in train_loader:
                inputs, targets = inputs.to(device), targets.to(device)

                optimizer.zero_grad()
                outputs = model(inputs)

                loss = criterion(outputs, targets, inputs[:,:,4].reshape(-1,80,1))
                loss.backward()
                optimizer.step()
                train_loss.append(loss.item())
            train_loss = np.mean(train_loss) 

            model.eval()
            with torch.no_grad():
                val_loss = []
                for inputs, targets in val_loader:
                    inputs, targets = inputs.to(device), targets.to(device)
                    outputs = model(inputs)
                    loss = criterion(outputs, targets, inputs[:,:,4].reshape(-1,80,1))
                    val_loss.append(loss.item())


                val_loss = np.mean(val_loss)


                if val_loss > 10:
                    model.load_state_dict(torch.load('/kaggle/working/ckpt_pytorch_ep_1'))
                    model.to(device)
                elif val_loss < best_loss:
                    best_loss = val_loss
                    torch.save(model.state_dict(), '/kaggle/working/ckpt_pytorch')
                if epoch==0:
                    torch.save(model.state_dict(), '/kaggle/working/ckpt_pytorch_ep_1')
        #         auc_roc = metric(outputs,targets.int()).compute()
                dt = datetime.now() - t0
                print(f'Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, \
          Val Loss: {val_loss:.4f} ,lowest loss {best_loss:.5f} ,    Duration: {dt}')


        with torch.no_grad():
            del model
            del inputs
            del targets
            del xtrain
            del xval
            del ytrain
            del yval
            del train_dataset
            del val_dataset
            del train_loader
            del val_loader
            gc.collect()
            with torch.cuda.device('cuda:0'):
                torch.cuda.empty_cache()
            model =LSTMModel(6,128,3)
            model.load_state_dict(torch.load('/kaggle/working/ckpt_pytorch'))
            model.to(device)
            preds = []
            model.eval()
            for inputs in test_loader:
                inputs = inputs.to(device)
                preds.append(model(inputs).reshape(-1,1)/kfold.n_splits)
            y_pred += torch.cat(preds,0)

    return y_pred.cpu().numpy() 


In [ ]:

y_pred=batch_gd(X, X_test, y, epochs=120, n_folds=5)

In [ ]:

sub = pd.read_csv('/kaggle/input/ventilator-pressure-prediction/sample_submission.csv')
sub.iloc[:,1]=y_pred
sub=sub.set_index('id')
sub.to_csv('baseline_pytorch_cv.csv')